In [1]:
import os
os.environ['CONDA_DEFAULT_ENV'] = '/Users/jafarabdurrahmaan/opt/anaconda3/envs/venv'

In [2]:
!conda activate venv


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [3]:
!conda info


     active environment : None
       user config file : /Users/jafarabdurrahmaan/.condarc
 populated config files : /Users/jafarabdurrahmaan/.condarc
          conda version : 4.10.3
    conda-build version : 3.21.5
         python version : 3.9.7.final.0
       virtual packages : __osx=10.16=0
                          __unix=0=0
                          __archspec=1=x86_64
       base environment : /Users/jafarabdurrahmaan/opt/anaconda3  (writable)
      conda av data dir : /Users/jafarabdurrahmaan/opt/anaconda3/etc/conda
  conda av metadata url : None
           channel URLs : https://conda.anaconda.org/conda-forge/osx-64
                          https://conda.anaconda.org/conda-forge/noarch
                          https://repo.anaconda.com/pkgs/main/osx-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/osx-64
                          https://repo.anaconda.com/pkgs/r/noarch
          package cach

In [4]:
!pip install couchbase

In [5]:
import couchbase

Insert data

In [6]:
from couchbase.cluster import Cluster, ClusterOptions
from couchbase.auth import PasswordAuthenticator
from couchbase.options import ClusterOptions
from datetime import datetime
import gzip
import json
import time

# Connect to the Couchbase cluster
cluster = Cluster('couchbase://localhost', ClusterOptions(
    authenticator=PasswordAuthenticator('smartmeterbenchmark', 'smartmeterbenchmark')
))
bucket = cluster.bucket("smartmeterbenchmark_c")
collection = bucket.default_collection()

# Open the bucket

# Get all document keys
result = cluster.query('SELECT META().id AS id FROM smartmeterbenchmark_c')
keys = [row['id'] for row in result]

# Delete each document by key
for key in keys:
    collection.remove(key)

In [7]:
from couchbase.cluster import Cluster, ClusterOptions
from couchbase.auth import PasswordAuthenticator
from couchbase.options import ClusterOptions
from datetime import datetime
import gzip
import json
import time

# Connect to cluster
cluster = Cluster.connect(
    "couchbase://localhost",
    ClusterOptions(PasswordAuthenticator("smartmeterbenchmark", "smartmeterbenchmark")))

# Access bucket and collection
bucket = cluster.bucket("smartmeterbenchmark_c")
collection = bucket.default_collection()

# Read JSON file
with open("/Users/jafarabdurrahmaan/Dropbox/00_Uni/Masterthesis/A_Thesis/1_Components/13_Benchmark_Study_Components/4_Document/Couchbase/iter2/daily_dataset_small.json") as f:
    data = json.load(f)

# Start timer
start_time = time.time()

# Insert each document into collection
for document in data:
    document["LCLid"] = str(document["LCLid"])
    document["day"] = datetime.strptime(document["day"], "%Y-%m-%d").strftime("%Y-%m-%d")
    document["energy_median"] = float(document["energy_median"])
    document["energy_mean"] = float(document["energy_mean"])
    document["energy_max"] = float(document["energy_max"])
    document["energy_count"] = int(document["energy_count"])
    document["energy_std"] = float(document["energy_std"])
    document["energy_sum"] = float(document["energy_sum"])
    document["energy_min"] = float(document["energy_min"])

    collection.insert(str(document["id"]), document)

# End timer
end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time

print("Data inserted successfully.")
print(f"Data insertion took: {execution_time} seconds.")

Data inserted successfully.
Data insertion took: 10.043436050415039 seconds.


In [ ]:
#Create index first before using N1QL query
from couchbase.exceptions import CouchbaseException

try:
    cluster.query_indexes().create_primary_index('smartmeterbenchmark_c')

except CouchbaseException as ex:
    import traceback
    traceback.print_exc()

Simple Retrieval 1

In [12]:
# Define the N1QL query
query = f"SELECT energy_min, energy_max FROM `smartmeterbenchmark_c` WHERE LCLid = 'MAC000131' AND day >= '2012-06-01' AND day <= '2013-06-01' LIMIT 5"

start_time = time.time()

# Execute the query
result = cluster.query(query)

end_time = time.time()
execution_time = (end_time - start_time)*1000

# Process the query results
for row in result:
    energy_min = row["energy_min"]
    energy_max = row["energy_max"]
    print(f"energy_min: {energy_min}, energy_max: {energy_max}")

print(f"Query execution time: {execution_time} milliseconds")

energy_min: 0.064, energy_max: 0.312
energy_min: 0.065, energy_max: 0.299
energy_min: 0.064, energy_max: 0.7829999999999999
energy_min: 0.066, energy_max: 1.1619999
energy_min: 0.065, energy_max: 0.742
Query execution time: 0.14591217041015625 milliseconds


Simple Retrieval 2

In [13]:
# Define the N1QL query
query = f"SELECT energy_min, energy_max FROM `smartmeterbenchmark_c` WHERE LCLid = 'MAC000131' AND day >= '2012-06-01' AND day <= '2013-06-01' LIMIT 5"

start_time = time.time()

# Execute the query
result = cluster.query(query)

end_time = time.time()
execution_time = (end_time - start_time)*1000

# Process the query results
for row in result:
    energy_min = row["energy_min"]
    energy_max = row["energy_max"]
    print(f"energy_min: {energy_min}, energy_max: {energy_max}")

print(f"Query execution time: {execution_time} milliseconds")

energy_min: 0.064, energy_max: 0.312
energy_min: 0.065, energy_max: 0.299
energy_min: 0.064, energy_max: 0.7829999999999999
energy_min: 0.066, energy_max: 1.1619999
energy_min: 0.065, energy_max: 0.742
Query execution time: 0.148773193359375 milliseconds


Simple Retrieval 3

In [14]:
# Define the N1QL query
query = f"SELECT energy_min, energy_max FROM `smartmeterbenchmark_c` WHERE LCLid = 'MAC000131' AND day >= '2012-06-01' AND day <= '2013-06-01' LIMIT 5"

start_time = time.time()

# Execute the query
result = cluster.query(query)

end_time = time.time()
execution_time = (end_time - start_time)*1000

# Process the query results
for row in result:
    energy_min = row["energy_min"]
    energy_max = row["energy_max"]
    print(f"energy_min: {energy_min}, energy_max: {energy_max}")

print(f"Query execution time: {execution_time} milliseconds")

energy_min: 0.064, energy_max: 0.312
energy_min: 0.065, energy_max: 0.299
energy_min: 0.064, energy_max: 0.7829999999999999
energy_min: 0.066, energy_max: 1.1619999
energy_min: 0.065, energy_max: 0.742
Query execution time: 0.16307830810546875 milliseconds


In [16]:
print(f"Average Query Execution Time: {((0.15+0.15+0.16)/3)} milliseconds")

Average Query Execution Time: 0.15333333333333332 milliseconds


Filter and Aggregation 1

In [23]:
# Define the N1QL query
query = f"SELECT LCLid, SUM(energy_sum) AS energy_sum FROM `smartmeterbenchmark_c` WHERE day >= '2012-06-01' AND day <= '2013-06-01' GROUP BY LCLid ORDER BY LCLid"

start_time = time.time()

# Execute the query
result = cluster.query(query)

end_time = time.time()
execution_time = (end_time - start_time)*1000

# Process the query results
for row in result:
    LCLid = row["LCLid"]
    energy_sum = row["energy_sum"]
    print(f"LCLid: {LCLid}, Sum of energy_sum: {energy_sum}")

print(f"Query execution time: {execution_time} milliseconds")

LCLid: MAC000131, Sum of energy_sum: 3691.994999699999
LCLid: MAC000132, Sum of energy_sum: 5258.847001099997
LCLid: MAC000221, Sum of energy_sum: 4917.545999700001
LCLid: MAC000228, Sum of energy_sum: 2707.6260002999993
LCLid: MAC000234, Sum of energy_sum: 4213.725000000001
LCLid: MAC000235, Sum of energy_sum: 1528.1550000999991
Query execution time: 0.13589859008789062 milliseconds


Filter and Aggregation 2

In [24]:
# Define the N1QL query
query = f"SELECT LCLid, SUM(energy_sum) AS energy_sum FROM `smartmeterbenchmark_c` WHERE day >= '2012-06-01' AND day <= '2013-06-01' GROUP BY LCLid ORDER BY LCLid"

start_time = time.time()

# Execute the query
result = cluster.query(query)

end_time = time.time()
execution_time = (end_time - start_time)*1000

# Process the query results
for row in result:
    LCLid = row["LCLid"]
    energy_sum = row["energy_sum"]
    print(f"LCLid: {LCLid}, Sum of energy_sum: {energy_sum}")

print(f"Query execution time: {execution_time} milliseconds")

LCLid: MAC000131, Sum of energy_sum: 3691.994999699999
LCLid: MAC000132, Sum of energy_sum: 5258.847001099997
LCLid: MAC000221, Sum of energy_sum: 4917.545999700001
LCLid: MAC000228, Sum of energy_sum: 2707.6260002999993
LCLid: MAC000234, Sum of energy_sum: 4213.725000000001
LCLid: MAC000235, Sum of energy_sum: 1528.1550000999991
Query execution time: 0.1518726348876953 milliseconds


Filter and Aggregation 3

In [25]:
# Define the N1QL query
query = f"SELECT LCLid, SUM(energy_sum) AS energy_sum FROM `smartmeterbenchmark_c` WHERE day >= '2012-06-01' AND day <= '2013-06-01' GROUP BY LCLid ORDER BY LCLid"

start_time = time.time()

# Execute the query
result = cluster.query(query)

end_time = time.time()
execution_time = (end_time - start_time)*1000

# Process the query results
for row in result:
    LCLid = row["LCLid"]
    energy_sum = row["energy_sum"]
    print(f"LCLid: {LCLid}, Sum of energy_sum: {energy_sum}")

print(f"Query execution time: {execution_time} milliseconds")

LCLid: MAC000131, Sum of energy_sum: 3691.994999699999
LCLid: MAC000132, Sum of energy_sum: 5258.847001099997
LCLid: MAC000221, Sum of energy_sum: 4917.545999700001
LCLid: MAC000228, Sum of energy_sum: 2707.6260002999993
LCLid: MAC000234, Sum of energy_sum: 4213.725000000001
LCLid: MAC000235, Sum of energy_sum: 1528.1550000999991
Query execution time: 0.13303756713867188 milliseconds


In [27]:
print(f"Average Query Execution Time: {((0.14+0.15+0.13)/3)} milliseconds")

Average Query Execution Time: 0.14 milliseconds
